# Unitary test of HOALS and comparison with HOSVD

In [1]:
from RecommenderSystem.HOALS import HOALS
from RecommenderSystem.metrics import rank_measure
from RecommenderSystem.utils import crossval_split
from sktensor.dtensor import dtensor
from sktensor.tucker import hosvd # baseline
import numpy as np
import pandas as pd

In [2]:
nUser = 10
nItem = 8
nAction = 3
ranks = [3,3,2]

np.random.seed(0)

U = np.random.random(nUser*ranks[0]).reshape((nUser,ranks[0]))
I = np.random.random(nItem*ranks[1]).reshape((nItem,ranks[1]))
A = np.random.random(nAction*ranks[2]).reshape((nAction,ranks[2]))
W = dtensor(np.random.random(np.prod(ranks)).reshape(tuple(ranks)))
T = W.ttm(U,mode=0).ttm(I,mode=1).ttm(A,mode=2)

In [3]:
u, i, a = (0,0,0)
print(W.ttv(U[u,:],modes=0).T.dot(I[i,:]).dot(A[a,:]))
print(T[u,i,a])

0.47708834093387403
0.47708834093387403


In [4]:
fulldata = pd.DataFrame(columns=['user','item','action','rating'])
for u in range(nUser):
    for i in range(nItem):
        for a in range(nAction):
            fulldata.loc[len(fulldata)] = [u,i,a,T[u,i,a]]
            
fulldata['user'] = list(map(int,fulldata['user']))
fulldata['item'] = list(map(int,fulldata['item']))
fulldata['action'] = list(map(int,fulldata['action']))

trainingData, testData = crossval_split(fulldata,0.8,0.7,'user','item')

## Train HOALS

In [5]:
sc.setCheckpointDir("checkpoints")
hoals = HOALS(ranks,lbda=0.01,alpha=0.0,max_iter=10)
hoals.fit(trainingData,True)
T_hat = hoals.getFullTensor()
rmse_hoals = np.sqrt(((T_hat-T)**2).mean())
print('RMSE of HOALS=',rmse_hoals)

	 Start user learning
	 	 time : 6.9243364334106445
	 Start item learning
	 	 time : 2.0978829860687256
	 Start action learning
	 	 time : 2.0889015197753906
	 	 longest mode time : 6.9243364334106445
RMSE of HOALS= 0.06416471050896164


** Compute rank measure **

In [6]:
indexes = testData[['user','item','action']].values.tolist()
r_hat = sc.parallelize(indexes).map(hoals.predict).collect()
testData['r_hat_hoals'] = r_hat

rank_perf_hoals = dict.fromkeys(range(nAction))
for action in range(nAction):
    testData_action = testData.loc[testData['action']==action,]
    rs = list(testData_action.sort_values(by='r_hat_hoals',ascending=False).groupby('user')['rating'].apply(list))
    rank_perf_hoals[action] = np.round(100*rank_measure(rs),2)
    print("Rank measure of action",action,":",rank_perf_hoals[action],"%")

Rank measure of action 0 : 15.5 %
Rank measure of action 1 : 15.04 %
Rank measure of action 2 : 15.22 %


## Train HOSVD

### on full tensor

In [7]:
baseline_hosvd_full = hosvd(T,ranks)
T_hat_hosvd_full = baseline_hosvd_full[1].ttm(baseline_hosvd_full[0][0],mode=0) \
                .ttm(baseline_hosvd_full[0][1],mode=1) \
                .ttm(baseline_hosvd_full[0][2],mode=2)
rmse_hosvd_full = np.sqrt(((T_hat_hosvd_full-T)**2).mean())
print('Full tensor - RMSE of HOSVD=',rmse_hosvd_full)

Full tensor - RMSE of HOSVD= 8.313589530222607e-15


** Compute rank measure **

In [8]:
testData['r_hat_hosvd_full'] = 0
for ind in testData.index:
    coordinates = tuple(list(map(int,testData.loc[ind,['user','item','action']])))
    testData.loc[ind,'r_hat_hosvd_full'] = T_hat_hosvd_full[coordinates]
    
rank_perf_hosvd_full = dict.fromkeys(range(nAction))
for action in range(nAction):
    testData_action = testData.loc[testData['action']==action,]
    rs = list(testData_action.sort_values(by='r_hat_hosvd_full',ascending=False).groupby('user')['rating'].apply(list))
    rank_perf_hosvd_full[action] = np.round(100*rank_measure(rs),2)
    print("Rank measure of action",action,":",rank_perf_hosvd_full[action],"%")

Rank measure of action 0 : 15.5 %
Rank measure of action 1 : 15.04 %
Rank measure of action 2 : 15.22 %


### on sparse tensor

In [9]:
baseline_hosvd_sparse = hosvd(dtensor(hoals.tensor.toarray()),ranks)
T_hat_hosvd_sparse = baseline_hosvd_sparse[1].ttm(baseline_hosvd_sparse[0][0],mode=0) \
                .ttm(baseline_hosvd_sparse[0][1],mode=1) \
                .ttm(baseline_hosvd_sparse[0][2],mode=2)
rmse_hosvd_sparse = np.sqrt(((T_hat_hosvd_sparse-T)**2).mean())
print('Sparse tensor - RMSE of HOSVD=',rmse_hosvd_sparse)

Sparse tensor - RMSE of HOSVD= 0.16408116651902568


In [10]:
testData['r_hat_hosvd_sparse'] = 0
for ind in testData.index:
    coordinates = tuple(list(map(int,testData.loc[ind,['user','item','action']])))
    testData.loc[ind,'hosvd_sparse'] = T_hat_hosvd_sparse[coordinates]
    
rank_perf_hosvd_sparse = dict.fromkeys(range(nAction))
for action in range(nAction):
    testData_action = testData.loc[testData['action']==action,]
    rs = list(testData_action.sort_values(by='r_hat_hosvd_sparse',ascending=False).groupby('user')['rating'].apply(list))
    rank_perf_hosvd_sparse[action] = np.round(100*rank_measure(rs),2)
    print("Rank measure of action",action,":",rank_perf_hosvd_sparse[action],"%")

Rank measure of action 0 : 16.03 %
Rank measure of action 1 : 16.09 %
Rank measure of action 2 : 16.07 %


** Compute rank measure **

### Print all results in the same cell

In [11]:
print('Full tensor - RMSE of HOSVD =',rmse_hosvd_full)
print('Sparse tensor - RMSE of HOSVD =',rmse_hosvd_sparse)
print('RMSE of HOALS =',rmse_hoals)

Full tensor - RMSE of HOSVD = 8.313589530222607e-15
Sparse tensor - RMSE of HOSVD = 0.16408116651902568
RMSE of HOALS = 0.06416471050896164


In [12]:
print('Full tensor - Rank measure of HOSVD =',rank_perf_hosvd_full)
print('Sparse tensor - Rank measure of HOSVD =',rank_perf_hosvd_sparse)
print('Rank measure of HOALS =',rank_perf_hoals)

Full tensor - Rank measure of HOSVD = {0: 15.5, 1: 15.04, 2: 15.22}
Sparse tensor - Rank measure of HOSVD = {0: 16.03, 1: 16.09, 2: 16.07}
Rank measure of HOALS = {0: 15.5, 1: 15.04, 2: 15.22}
